## Overview
The goal of the UBC Ovarian Cancer subtypE clAssification and outlier detectioN (UBC-OCEAN) competition is to classify ovarian cancer subtypes. You will build a model trained on the world's most extensive ovarian cancer dataset of histopathology images obtained from more than 20 medical centers.

Your work will help enhance the applicability and accessibility of accurate ovarian cancer diagnoses.

## Description

Ovarian carcinoma is the most lethal cancer of the female reproductive system. There are five common subtypes of ovarian cancer: high-grade serous carcinoma, clear-cell ovarian carcinoma, endometrioid, low-grade serous, and mucinous carcinoma. Additionally, there are several rare subtypes ("Outliers"). These are all characterized by distinct cellular morphologies, etiologies, molecular and genetic profiles, and clinical attributes. Subtype-specific treatment approaches are gaining prominence, though first requires subtype identification, a process that could be improved with data science.

Currently, ovarian cancer diagnosis relies on pathologists to assess subtypes. However, this presents several challenges, including disagreements between observers and the reproducibility of diagnostics. Furthermore, underserved communities often lack access to specialist pathologists, and even well-developed communities face a shortage of pathologists with expertise in gynecologic malignancies.

Deep learning models have exhibited remarkable proficiency in analyzing histopathology images. Yet challenges still exist, such as the need for a significant amount of training data, ideally from a single source. Technical, ethical, and financial constraints, as well as confidentiality concerns, make training a challenge. In this competition, you will have access to the most extensive and diverse ovarian cancer dataset of histopathology images from more than 20 centers across four continents.

![ubc1](https://storage.googleapis.com/kaggle-media/competitions/UBC/OCEAN-Optional-Figure.png)

Competition host University of British Columbia (UBC) is a global center for teaching, learning, and research, consistently ranked among the top 20 public universities in the world. UBC embraces innovation and transforms ideas into action. Since 1915, UBC has been opening doors of opportunity for people with the curiosity, drive, and vision to shape a better world. Joining UBC is the BC Cancer, part of the Provincial Health Services Authority and its world-renowned Ovarian Cancer Research (OVCARE) team whose discoveries have led to progressive prevention strategies and improved diagnostics and treatments. BC Cancer provides a comprehensive cancer control program for the people of British Columbia in partnership with regional health authorities. We have also partnered with the Ovarian Tumour Tissue Analysis (OTTA) consortium, an international multidisciplinary network of investigators from more than 65 international teams across the globe. Finally, the OCEAN challenge is made possible through a generous donation from TD Bank Group through the BC Cancer Foundation.

Your work could yield improved accuracy in identifying ovarian cancer subtypes. Better classification would enable clinicians to formulate personalized treatment strategies regardless of geographic location. This targeted approach has the potential to enhance treatment efficacy, reduce adverse effects, and ultimately contribute to better patient outcomes for those diagnosed with this deadly cancer.

Using the data outside of the competition

We request that participants refrain from utilizing the competition data, either in its entirety or in part, for any external projects, research, or applications until the official publication of the competition paper. We will ensure that participants receive prompt notification when the embargo is lifted.

In [1]:
import os
import cv2
import copy
import time
import random
from PIL import Image
from IPython.display import clear_output
from numba import cuda

# For data manipulation
import numpy as np
import pandas as pd

# Pytorch Imports
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader
from torch.cuda import amp
import torchvision
import torchvision.models as models
from torch.optim import Optimizer

# Utils
from tqdm import tqdm
from collections import defaultdict

# Sklearn Imports
from sklearn.model_selection import train_test_split

# For Image Models
import timm

# Albumentations for augmentations
import albumentations as A
from albumentations.pytorch import ToTensorV2

import warnings
warnings.filterwarnings("ignore")

# For descriptive error messages
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
CONFIG = {
    "seed": 2023,
    "img_size": 2048,
    "num_classes": 5,
    "batch": 1,
    "device": torch.device("cuda:0" if torch.cuda.is_available() else "cpu"),
    "lr": 0.0005,
    "Threshold" : 0.3
}
training=False

In [3]:
ROOT_DIR = '/kaggle/input/UBC-OCEAN'
TEST_DIR = '/kaggle/input/UBC-OCEAN/test_thumbnails'
ALT_TEST_DIR = '/kaggle/input/UBC-OCEAN/test_images'
Train_DIR = '/kaggle/input/UBC-OCEAN/train_thumbnails'
ALT_Train_DIR = '/kaggle/input/UBC-OCEAN/train_images'

In [4]:
def get_test_file_path(image_id):
    if os.path.exists(f"{TEST_DIR}/{image_id}_thumbnail.png"):
        return str(f"{TEST_DIR}/{image_id}_thumbnail.png")
    else:
        return str(f"{ALT_TEST_DIR}/{image_id}.png")
    
def get_train_file_path(image_id):
    if os.path.exists(f"{Train_DIR}/{image_id}_thumbnail.png"):
        return str(f"{Train_DIR}/{image_id}_thumbnail.png")
    else:
        return str(f"{ALT_Train_DIR}/{image_id}.png")
    #return str(f"{ALT_Train_DIR}/{image_id}.png")

In [5]:
test_df = pd.read_csv(f"{ROOT_DIR}/test.csv")
test_df['file_path'] = test_df['image_id'].apply(get_test_file_path)
test_df['label'] = 0 # dummy
test_df

,image_id,image_width,image_height,file_path,label
0,41,28469,16987,/kaggle/input/UBC-OCEAN/test_thumbnails/41_thu...,0


In [6]:
df_sub = pd.read_csv(f"{ROOT_DIR}/sample_submission.csv")
df_sub

,image_id,label
0,41,HGSC


In [7]:
train_df = pd.read_csv("/kaggle/input/UBC-OCEAN/train.csv")
train_df['file_path'] = train_df['image_id'].apply(get_train_file_path)
print(train_df['label'].unique())
train_df['label_name']=train_df['label']
train_df['label'] = train_df['label'].astype('category').cat.codes
label_mapping = dict(enumerate(train_df['label_name'].astype('category').cat.categories))
label_mapping[5] = 'Others'
print(label_mapping)
train_df

['HGSC' 'LGSC' 'EC' 'CC' 'MC']
{0: 'CC', 1: 'EC', 2: 'HGSC', 3: 'LGSC', 4: 'MC', 5: 'Others'}


,image_id,label,image_width,image_height,is_tma,file_path,label_name
0,4,2,23785,20008,False,/kaggle/input/UBC-OCEAN/train_thumbnails/4_thu...,HGSC
1,66,3,48871,48195,False,/kaggle/input/UBC-OCEAN/train_thumbnails/66_th...,LGSC
2,91,2,3388,3388,True,/kaggle/input/UBC-OCEAN/train_images/91.png,HGSC
3,281,3,42309,15545,False,/kaggle/input/UBC-OCEAN/train_thumbnails/281_t...,LGSC
4,286,1,37204,30020,False,/kaggle/input/UBC-OCEAN/train_thumbnails/286_t...,EC
...,...,...,...,...,...,...,...
533,65022,3,53355,46675,False,/kaggle/input/UBC-OCEAN/train_thumbnails/65022...,LGSC
534,65094,4,55042,45080,False,/kaggle/input/UBC-OCEAN/train_thumbnails/65094...,MC
535,65300,2,75860,27503,False,/kaggle/input/UBC-OCEAN/train_thumbnails/65300...,HGSC
536,65371,2,42551,41800,False,/kaggle/input/UBC-OCEAN/train_thumbnails/65371...,HGSC


In [8]:
def get_cropped_images(file_path, image_id, label_t,th_area = 1000):
    image = Image.open(file_path)
    # Aspect ratio
    as_ratio = image.size[0] / image.size[1]
    
    sxs, exs, sys, eys = [],[],[],[]
    if as_ratio >= 1.5:
        # Crop
        mask = np.max( np.array(image) > 0, axis=-1 ).astype(np.uint8)
        retval, labels = cv2.connectedComponents(mask)
        if retval >= as_ratio:
            x, y = np.meshgrid( np.arange(image.size[0]), np.arange(image.size[1]) )
            for label in range(1, retval):
                area = np.sum(labels == label)
                if area < th_area:
                    continue
                xs, ys= x[ labels == label ], y[ labels == label ]
                sx, ex = np.min(xs), np.max(xs)
                cx = (sx + ex) // 2
                crop_size = image.size[1]
                sx = max(0, cx-crop_size//2)
                ex = min(sx + crop_size - 1, image.size[0]-1)
                sx = ex - crop_size + 1
                sy, ey = 0, image.size[1]-1
                sxs.append(sx)
                exs.append(ex)
                sys.append(sy)
                eys.append(ey)
        else:
            crop_size = image.size[1]
            for i in range(int(as_ratio)):
                sxs.append( i * crop_size )
                exs.append( (i+1) * crop_size - 1 )
                sys.append( 0 )
                eys.append( crop_size - 1 )
    else:
        # Not Crop (entire image)
        sxs, exs, sys, eys = [0,],[image.size[0]-1],[0,],[image.size[1]-1]

    df_crop = pd.DataFrame()
    df_crop["image_id"] = [image_id] * len(sxs)
    df_crop["file_path"] = [file_path] * len(sxs)
    df_crop["sx"] = sxs
    df_crop["ex"] = exs
    df_crop["sy"] = sys
    df_crop["ey"] = eys
    df_crop["label"] = [label_t] * len(sxs)
    return df_crop

In [9]:
# Assuming train_df is your DataFrame and 'label' is the column with class labels
labels = train_df['label'].value_counts()
print(labels)

# Calculate total number of samples
total_samples = np.sum(labels)

# Number of classes
num_classes = len(labels)

# Calculating weights for each class
weight_balance = {}
for label, count in labels.items():
    weight_balance[label] = total_samples / (num_classes * count)

print(weight_balance)

label
2    222
1    124
0     99
3     47
4     46
Name: count, dtype: int64
{2: 0.4846846846846847, 1: 0.867741935483871, 0: 1.0868686868686868, 3: 2.2893617021276595, 4: 2.3391304347826085}


In [10]:
'''train_df_tma = train_df[train_df['is_tma']==True]
train_df_no_tma = train_df[train_df['is_tma']==False]'''

"train_df_tma = train_df[train_df['is_tma']==True]\ntrain_df_no_tma = train_df[train_df['is_tma']==False]"

In [11]:
class UBCDataset(Dataset):
    def __init__(self, df, transforms=None):
        self.df = df.reset_index(drop=True)  # Reset index
        self.file_names = self.df['file_path'].values
        self.labels = self.df['label'].values
        self.transforms = transforms
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        img_path = self.file_names[index]
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        label = self.labels[index]
        
        if self.transforms:
            img = self.transforms(image=img)["image"]
            
        img = torch.tensor(img, dtype=torch.float)  # Convert image to torch tensor
        label = torch.tensor(label, dtype=torch.long)  # Ensure label is a long tensor
        
        return img, label  # Return image and label as a tuple

class UBCDataset2(Dataset):
    def __init__(self, df, transforms=None):
        self.df = df
        self.file_names = df['file_path'].values
        self.labels = df['label'].values
        self.transforms = transforms
        self.sxs = df["sx"].values
        self.exs = df["ex"].values
        self.sys = df["sy"].values
        self.eys = df["ey"].values
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        img_path = self.file_names[index]
        sx = self.sxs[index]
        ex = self.exs[index]
        sy = self.sys[index]
        ey = self.eys[index]
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        label = self.labels[index]
        
        img = img[ sy:ey, sx:ex, : ]
        
        if self.transforms:
            img = self.transforms(image=img)["image"]
            
        img = torch.tensor(img, dtype=torch.float)  # Convert image to torch tensor
        label = torch.tensor(label, dtype=torch.long)  # Ensure label is a long tensor
        
        return img, label  # Return image and label as a tuple


In [12]:
data_transforms = {
    "valid": A.Compose([
        A.Resize(CONFIG['img_size'], CONFIG['img_size']),
        A.Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225],
                max_pixel_value=255.0,
                p=1.0
            ),
        ToTensorV2()], p=1.)
}

data_transforms["O"] = A.Compose([
    A.Resize(CONFIG['img_size'], CONFIG['img_size']),
    A.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225],
        max_pixel_value=255.0,
        p=1.0
    ),
    ToTensorV2()], p=1.)

data_transforms["A"] = A.Compose([
    A.Resize(CONFIG['img_size'], CONFIG['img_size']),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.75),
    A.ShiftScaleRotate(p=0.75),
    A.OneOf([
    A.GaussNoise(var_limit=[10, 50]),
    A.GaussianBlur(),
    A.MotionBlur(),
    ], p=0.4),
    A.GridDistortion(num_steps=5, distort_limit=0.3, p=0.5),
    A.CoarseDropout(max_holes=1, max_width=int(CONFIG['img_size']* 0.3), max_height=int(CONFIG['img_size']* 0.3),
    mask_fill_value=0, p=0.5),
    A.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225],
        max_pixel_value=255.0,
        p=1.0
    ),
    ToTensorV2()], p=1.)

'''data_transforms["OR"] = A.Compose([
    A.Resize(CONFIG['img_size'], CONFIG['img_size']),
    A.RandomResizedCrop(int(CONFIG['img_size']), int(CONFIG['img_size'])),
    A.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225],
        max_pixel_value=255.0,
        p=1.0
    ),
    ToTensorV2()], p=1.)

data_transforms["AR"] = A.Compose([
    A.Resize(CONFIG['img_size'], CONFIG['img_size']),
    A.RandomResizedCrop(int(CONFIG['img_size']), int(CONFIG['img_size'])),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.75),
    A.ShiftScaleRotate(p=0.75),
    A.OneOf([
    A.GaussNoise(var_limit=[10, 50]),
    A.GaussianBlur(),
    A.MotionBlur(),
    ], p=0.4),
    A.GridDistortion(num_steps=5, distort_limit=0.3, p=0.5),
    A.CoarseDropout(max_holes=1, max_width=int(CONFIG['img_size']* 0.3), max_height=int(CONFIG['img_size']* 0.3),
    mask_fill_value=0, p=0.5),
    A.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225],
        max_pixel_value=255.0,
        p=1.0
    ),
    ToTensorV2()], p=1.)'''

'data_transforms["OR"] = A.Compose([\n    A.Resize(CONFIG[\'img_size\'], CONFIG[\'img_size\']),\n    A.RandomResizedCrop(int(CONFIG[\'img_size\']), int(CONFIG[\'img_size\'])),\n    A.Normalize(\n        mean=[0.485, 0.456, 0.406],\n        std=[0.229, 0.224, 0.225],\n        max_pixel_value=255.0,\n        p=1.0\n    ),\n    ToTensorV2()], p=1.)\n\ndata_transforms["AR"] = A.Compose([\n    A.Resize(CONFIG[\'img_size\'], CONFIG[\'img_size\']),\n    A.RandomResizedCrop(int(CONFIG[\'img_size\']), int(CONFIG[\'img_size\'])),\n    A.HorizontalFlip(p=0.5),\n    A.VerticalFlip(p=0.5),\n    A.RandomBrightnessContrast(p=0.75),\n    A.ShiftScaleRotate(p=0.75),\n    A.OneOf([\n    A.GaussNoise(var_limit=[10, 50]),\n    A.GaussianBlur(),\n    A.MotionBlur(),\n    ], p=0.4),\n    A.GridDistortion(num_steps=5, distort_limit=0.3, p=0.5),\n    A.CoarseDropout(max_holes=1, max_width=int(CONFIG[\'img_size\']* 0.3), max_height=int(CONFIG[\'img_size\']* 0.3),\n    mask_fill_value=0, p=0.5),\n    A.Normaliz

In [13]:
  '''  def classify(self, images):
        with torch.no_grad():
            _, probs = self.forward(images)
            probabilities = torch.nn.functional.softmax(probs, dim=1)
            max_prob, preds = torch.max(probabilities, 1)
            classifications = torch.where(
                max_prob > self.threshold,
                preds,
                torch.full_like(preds, self.num_classes)  # 'Other' class
            )
            return classifications'''

"  def classify(self, images):\n      with torch.no_grad():\n          _, probs = self.forward(images)\n          probabilities = torch.nn.functional.softmax(probs, dim=1)\n          max_prob, preds = torch.max(probabilities, 1)\n          classifications = torch.where(\n              max_prob > self.threshold,\n              preds,\n              torch.full_like(preds, self.num_classes)  # 'Other' class\n          )\n          return classifications"

In [14]:
class GeM(nn.Module):
    def __init__(self, p=3, eps=1e-6):
        super(GeM, self).__init__()
        self.p = nn.Parameter(torch.ones(1)*p)
        self.eps = eps

    def forward(self, x):
        return self.gem(x, p=self.p, eps=self.eps)
        
    def gem(self, x, p=3, eps=1e-6):
        return F.avg_pool2d(x.clamp(min=eps).pow(p), (x.size(-2), x.size(-1))).pow(1./p)
        
    def __repr__(self):
        return self.__class__.__name__ + \
                '(' + 'p=' + '{:.4f}'.format(self.p.data.tolist()[0]) + \
                ', ' + 'eps=' + str(self.eps) + ')'
    
class EfficientNet(nn.Module):
    def __init__(self, num_classes=5, threshold=0.3, pretrained=True, checkpoint_path=None):
        super(EfficientNet, self).__init__()
        self.model = timm.create_model('tf_efficientnet_b0', checkpoint_path='/kaggle/input/tf-efficientnet/pytorch/tf-efficientnet-b0/1/tf_efficientnet_b0_aa-827b6e33.pth')
        # Keeping the feature extraction part and removing the classification head
        self.threshold = threshold
        self.num_classes = num_classes
        self.pooling = GeM()
        in_features = self.model.classifier.in_features
        self.model.classifier = nn.Identity()
        self.model.global_pool = nn.Identity()
        self.dropout = nn.Dropout(p=0.5)  # Adjust p as necessary
        self.batch_norm = nn.BatchNorm1d(in_features)  # Batch normalization
        self.linear = nn.Linear(in_features, num_classes)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, images):
        features = self.model(images)
        pooled_features = self.pooling(features).flatten(1)
        pooled_features = self.dropout(pooled_features)  # Dropout
        # Check the batch size and skip batch normalization if the batch size is 1
        if pooled_features.size(0) > 1:
            pooled_features = self.batch_norm(pooled_features)
        else:
            # Optionally print a warning or handle the singleton batch differently
            pass
        output = self.linear(pooled_features)
        probs = self.softmax(output)
        return output, probs

    def training_step(self, batch, batch_idx):
        images, labels = batch
        outputs = self(images)
        loss = F.cross_entropy(outputs, labels)
        return loss
        
    def classify(self, images):
        with torch.no_grad():
            _, probs = self.forward(images)
            max_prob, preds = torch.max(probs, 1)
            # Apply threshold to determine 'Other'
            classifications = torch.where(
                max_prob > self.threshold,
                preds,
                #torch.tensor(self.num_classes)
                torch.full_like(preds, self.num_classes)  # 'Other' class
            )
            return classifications

class EfficientNet2(nn.Module):
    def __init__(self, num_classes=5, threshold=0.3, pretrained=True, checkpoint_path=None):
        super(EfficientNet2, self).__init__()
        self.model = timm.create_model('tf_efficientnetv2_s_in21ft1k', pretrained=False)
        pretrained_dict=torch.load('/kaggle/input/effi-v2/v2s.pth')
        self.model.load_state_dict(pretrained_dict)
        # Keeping the feature extraction part and removing the classification head
        self.threshold = threshold
        self.num_classes = num_classes
        self.pooling = GeM()
        in_features = self.model.classifier.in_features
        self.model.classifier = nn.Identity()
        self.model.global_pool = nn.Identity()
        self.dropout = nn.Dropout(p=0.5)  # Adjust p as necessary
        self.batch_norm = nn.BatchNorm1d(in_features)  # Batch normalization
        self.linear = nn.Linear(in_features, num_classes)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, images):
        features = self.model(images)
        pooled_features = self.pooling(features).flatten(1)
        pooled_features = self.dropout(pooled_features)  # Dropout
        # Check the batch size and skip batch normalization if the batch size is 1
        if pooled_features.size(0) > 1:
            pooled_features = self.batch_norm(pooled_features)
        else:
            # Optionally print a warning or handle the singleton batch differently
            pass
        output = self.linear(pooled_features)
        probs = self.softmax(output)
        return output, probs

    def training_step(self, batch, batch_idx):
        images, labels = batch
        outputs = self(images)
        loss = F.cross_entropy(outputs, labels)
        return loss
        
    def classify(self, images):
        with torch.no_grad():
            _, probs = self.forward(images)
            max_prob, preds = torch.max(probs, 1)
            # Apply threshold to determine 'Other'
            classifications = torch.where(
                max_prob > self.threshold,
                preds,
                #torch.tensor(self.num_classes)
                torch.full_like(preds, self.num_classes)  # 'Other' class
            )
            return classifications        
        

class SAM(Optimizer):
    def __init__(self, params, base_optimizer, rho=0.05, **kwargs):
        assert rho >= 0.0, f"Invalid rho, should be non-negative: {rho}"

        defaults = dict(rho=rho, **kwargs)
        super(SAM, self).__init__(params, defaults)

        self.base_optimizer = base_optimizer(self.param_groups, **kwargs)
        self.param_groups = self.base_optimizer.param_groups

    @torch.no_grad()
    def first_step(self, zero_grad=False):
        grad_norm = self._grad_norm()
        for group in self.param_groups:
            scale = group["rho"] / (grad_norm + 1e-12)
            for p in group["params"]:
                if p.grad is not None:
                    self.state[p]["e_w"] = p.grad * scale
                    p.add_(self.state[p]["e_w"])  # climb to the local maximum
        if zero_grad:
            self.zero_grad()

    @torch.no_grad()
    def second_step(self, zero_grad=False):
        for group in self.param_groups:
            for p in group["params"]:
                if p.grad is not None:
                    p.sub_(self.state[p]["e_w"])  # get back to the original parameters

        self.base_optimizer.step()  # do the actual "sharpness-aware" update

        if zero_grad:
            self.zero_grad()

    def step(self, closure=None):
        raise NotImplementedError("SAM doesn't work like the other optimizers, you should first call `first_step` and the `second_step`; see the documentation for more info.")

    def _grad_norm(self):
        shared_device = self.param_groups[0]["params"][0].device  # put everything on the same device, in case of model parallelism
        gradients = [
            p.grad.norm(p=2).to(shared_device)
            for group in self.param_groups for p in group["params"]
            if p.grad is not None
        ]

        if not gradients:  # If there are no gradients, return a dummy value (e.g., 0)
            return torch.tensor(0.0).to(shared_device)

        norm = torch.norm(torch.stack(gradients), p=2)
        return norm

#base_optimizer = torch.optim.SGD
#optimizer = SAM(model.parameters(), base_optimizer, lr=CONFIG["lr"], momentum=0.9)

In [15]:
'''from torch.utils.data import DataLoader, ConcatDataset, random_split

if training:
    # Concatenating the datasets
    #O_train_dataset = UBCDataset2(df_train, transforms=data_transforms['O'])
    A_train_dataset = UBCDataset(train_df, transforms=data_transforms['A'])
    #OR_train_dataset = UBCDataset2(df_train, transforms=data_transforms['OR'])
    #AR_train_dataset = UBCDataset2(df_train, transforms=data_transforms['AR'])

    combined_dataset = ConcatDataset([A_train_dataset])#, O_train_dataset, OR_train_dataset, AR_train_dataset])

    # Splitting the combined dataset into 5 parts
    total_size = len(combined_dataset)
    part_size = total_size // 5
    remainder = total_size % 5
    lengths = [part_size + (1 if i < remainder else 0) for i in range(5)]

    split_datasets = random_split(combined_dataset, lengths)

    train_dic = {}
    valid_dic = {}

    for i in range(5):
        temp_v = split_datasets[i]
        temp_t = [split_datasets[j] for j in range(5) if j != i]
        train_dic[f'v{i+1}'] = DataLoader(ConcatDataset(temp_t), batch_size=CONFIG['batch'], num_workers=2, shuffle=False, pin_memory=True)
        valid_dic[f'v{i+1}'] = DataLoader(temp_v, batch_size=CONFIG['batch'], num_workers=2, shuffle=False, pin_memory=True)'''

"from torch.utils.data import DataLoader, ConcatDataset, random_split\n\nif training:\n    # Concatenating the datasets\n    #O_train_dataset = UBCDataset2(df_train, transforms=data_transforms['O'])\n    A_train_dataset = UBCDataset(train_df, transforms=data_transforms['A'])\n    #OR_train_dataset = UBCDataset2(df_train, transforms=data_transforms['OR'])\n    #AR_train_dataset = UBCDataset2(df_train, transforms=data_transforms['AR'])\n\n    combined_dataset = ConcatDataset([A_train_dataset])#, O_train_dataset, OR_train_dataset, AR_train_dataset])\n\n    # Splitting the combined dataset into 5 parts\n    total_size = len(combined_dataset)\n    part_size = total_size // 5\n    remainder = total_size % 5\n    lengths = [part_size + (1 if i < remainder else 0) for i in range(5)]\n\n    split_datasets = random_split(combined_dataset, lengths)\n\n    train_dic = {}\n    valid_dic = {}\n\n    for i in range(5):\n        temp_v = split_datasets[i]\n        temp_t = [split_datasets[j] for j in 

In [16]:
from torch.utils.data import DataLoader, ConcatDataset, random_split
split=3
if training:
    # Initialize dictionaries to store the data loaders for each fold
    train_dic = {}
    valid_dic = {}

    # Manually splitting the DataFrame into 5 parts
    df_splits = np.array_split(train_df.sample(frac=1, random_state=42), split)

    # Iterate over each split to create combined datasets
    for i in range(split):
        # Collecting all parts except the ith for training
        train_df_split = pd.concat([df_splits[j] for j in range(split) if j != i]).reset_index(drop=True)

        # The ith part is used for validation
        valid_df_split = df_splits[i]
        
        dfs = []
        for (file_path, image_id,label) in zip(train_df_split["file_path"], train_df_split["image_id"],train_df_split['label']):
            dfs.append(get_cropped_images(file_path, image_id,label))

        df_train = pd.concat(dfs)
        df_train = df_train.drop_duplicates(subset=["image_id", "sx", "ex", "sy", "ey"]).reset_index(drop=True)
        
        dfs = []
        for (file_path, image_id,label) in zip(valid_df_split["file_path"], valid_df_split["image_id"],valid_df_split['label']):
            dfs.append( get_cropped_images(file_path, image_id,label) )

        df_crop = pd.concat(dfs)
        df_crop = df_crop.drop_duplicates(subset=["image_id", "sx", "ex", "sy", "ey"]).reset_index(drop=True)

        # Define the datasets with different transformations for the splits
        train_datasets = {
            #'A': UBCDataset2(df_train, transforms=data_transforms['A']),
            'O': UBCDataset2(df_train, transforms=data_transforms['O']),
            # Add other datasets as needed
        }

        valid_datasets = {
            'A': UBCDataset2(df_crop, transforms=data_transforms['valid']),
            # Add other datasets as needed
        }

        # Creating DataLoaders for the ith fold
        train_dic[f'v{i+1}'] = DataLoader(ConcatDataset(list(train_datasets.values())), batch_size=CONFIG['batch'], num_workers=2, shuffle=False, pin_memory=True)
        valid_dic[f'v{i+1}'] = DataLoader(ConcatDataset(list(valid_datasets.values())), batch_size=CONFIG['batch'], num_workers=2, shuffle=False, pin_memory=True)

In [17]:
if training:
    clear_output(wait=True)
    #device = cuda.get_current_device(); device.reset()
    torch.cuda.empty_cache()
    # Loss and optimizer
    class_weights = torch.Tensor(list(weight_balance.values())).to(CONFIG['device'])
    # Training loop
    for i in range(split):
        model = EfficientNet2(num_classes=CONFIG['num_classes'], threshold = CONFIG["Threshold"])
        model.to(CONFIG['device']);
        criterion = nn.CrossEntropyLoss(weight=class_weights)
        optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)#, weight_decay=0.3)
        base_optimizer = torch.optim.Adam
        optimizer2 = SAM(model.parameters(), base_optimizer, lr=CONFIG["lr"])
        best_loss = float('inf')
        obj=True
        epoch=0
        train_loss_dic={}
        valid_loss_dic={}
        while obj:  # Number of epochs
            train_loss = 0.0
            valid_loss = 0.0
            model.train()  # Set the model to training mode
            print(f'v{i+1}')
            for data in tqdm(train_dic[f'v{i+1}']):
                images, labels = data[0].to(CONFIG['device']), data[1].to(CONFIG['device'])
                # Forward pass
                classifications,_ = model(images)
                loss = criterion(classifications, labels)
                # Backward pass and optimization
                if True: #valid_loss == 0.0 or valid_loss > 1.2:
                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()
                else:
                    optimizer2.zero_grad()
                    loss.backward()
                    optimizer2.first_step(zero_grad=True)
                    criterion(model(images)[0], labels).backward()  # You can use outputs from the first forward pass 
                    optimizer2.second_step()
                train_loss+=loss.item() * len(data)
                
            train_loss = train_loss/len(train_dic[f'v{i+1}'].sampler)
            train_loss_dic[f't{epoch+1}']=train_loss
            # Validation loop
            model.eval()  # Set the model to evaluation mode
            valid_loss = 0
            with torch.no_grad():  # Disable gradient computation
                for data in tqdm(valid_dic[f'v{i+1}']):
                    images, labels = data[0].to(CONFIG['device']), data[1].to(CONFIG['device'])
                    classifications, _ = model(images)
                    v_loss = criterion(classifications, labels)
                    valid_loss += v_loss.item()
            valid_loss = valid_loss/len(valid_dic[f'v{i+1}'].sampler)
            valid_loss_dic[f't{epoch+1}']=valid_loss

            # Save the model if validation loss has decreased
            if valid_loss < best_loss:
                best_loss = valid_loss
                torch.save(model.state_dict(), f'best_model_weights{i+1}.pth')
                print('Best Model Saved')

            # Check for stopping condition
            if epoch == 0:
                cri=valid_loss*0.7
            if best_loss < cri:
                obj=False
            if epoch > 5 and valid_loss_dic[f't{epoch-1}']-valid_loss_dic[f't{epoch}'] < 0 and valid_loss_dic[f't{epoch}']-valid_loss_dic[f't{epoch+1}'] < 0 and valid_loss_dic[f't{epoch-2}']-valid_loss_dic[f't{epoch-1}'] < 0:
                obj=False
            epoch += 1
            print(f'Epoch [{epoch}], Training Loss: {train_loss:.4f}, Validation Loss: {valid_loss:.4f}, fold: {i+1}')

In [18]:
from collections import Counter
import numpy as np

'''
def most_frequent_element(arr):
    counter = Counter(arr)
    most_common = counter.most_common(1)[0][0]
    return most_common
model = EfficientNet(num_classes=CONFIG['num_classes'], threshold=CONFIG["Threshold"])
model.to(CONFIG['device']);

dfs = []
for (file_path, image_id,label) in zip(test_df["file_path"], test_df["image_id"],test_df['label']):
    dfs.append( get_cropped_images(file_path, image_id,label) )

df_crop = pd.concat(dfs)
df_crop["label"] = 0 # dummy

df_crop = df_crop.drop_duplicates(subset=["image_id", "sx", "ex", "sy", "ey"]).reset_index(drop=True)
test_dataset = UBCDataset2(df_crop, transforms=data_transforms['valid'])
test_loader = DataLoader(test_dataset, batch_size=CONFIG['batch'], num_workers=2, shuffle=False, pin_memory=True)

# Initialize a list to store predictions from all models
all_model_preds = []
all_model_prob = []
for i in range(5):
    model.load_state_dict(torch.load(f'/kaggle/input/weights4/best_model_weights{i+1}.pth',map_location=torch.device(CONFIG['device'])))
    model.eval()
    model_preds = []
    model_prob = []
    with torch.no_grad():
        for images, _ in test_loader:        
            images = images.to(CONFIG['device'], dtype=torch.float)

            _, probabilities = model(images)
            model_prob.append(probabilities)
            _, predicted = torch.max(probabilities, 1)
            model_preds.extend(predicted.detach().cpu().numpy())
    all_model_prob.append(model_prob)
    all_model_preds.append(model_preds)

# Transpose to align predictions for each data point across models
combined_pred = np.array(all_model_preds).T

# Find the most frequent predictions for each data point
aggregated_labels = [most_frequent_element(pred_list) for pred_list in combined_pred]

# Assuming label_mapping is defined to map labels to actual class names
pred_labels = [label_mapping[label] for label in aggregated_labels]

# Assuming df_sub is your submission dataframe
df_sub["label"] = pred_labels
df_sub.to_csv("submission.csv", index=False)

if training:
    print("Probabilities:", all_model_prob)
    print("Predictions:", combined_pred)
    print("Aggregated Labels:", pred_labels)'''

'\ndef most_frequent_element(arr):\n    counter = Counter(arr)\n    most_common = counter.most_common(1)[0][0]\n    return most_common\nmodel = EfficientNet(num_classes=CONFIG[\'num_classes\'], threshold=CONFIG["Threshold"])\nmodel.to(CONFIG[\'device\']);\n\ndfs = []\nfor (file_path, image_id,label) in zip(test_df["file_path"], test_df["image_id"],test_df[\'label\']):\n    dfs.append( get_cropped_images(file_path, image_id,label) )\n\ndf_crop = pd.concat(dfs)\ndf_crop["label"] = 0 # dummy\n\ndf_crop = df_crop.drop_duplicates(subset=["image_id", "sx", "ex", "sy", "ey"]).reset_index(drop=True)\ntest_dataset = UBCDataset2(df_crop, transforms=data_transforms[\'valid\'])\ntest_loader = DataLoader(test_dataset, batch_size=CONFIG[\'batch\'], num_workers=2, shuffle=False, pin_memory=True)\n\n# Initialize a list to store predictions from all models\nall_model_preds = []\nall_model_prob = []\nfor i in range(5):\n    model.load_state_dict(torch.load(f\'/kaggle/input/weights4/best_model_weights{i+

In [19]:
'''train_dataset = UBCDataset(train_df, transforms=data_transforms['N'])
train_loader = DataLoader(train_dataset, batch_size=CONFIG['batch'], num_workers=2, shuffle=False, pin_memory=True)


# Initialize model
model = EfficientNet2(num_classes=CONFIG['num_classes'], threshold=0.3)
#model = ViTModel(threshold=0.3)
model.to(CONFIG['device']);
clear_output(wait=True)

# Loss and optimizer
class_weights = torch.Tensor(list(weight_balance.values())).to(CONFIG['device'])

# Loss with class weights
criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = torch.optim.Adam(model.parameters(), lr=CONFIG["lr"])
#base_optimizer = torch.optim.Adam
#optimizer2 = SAM(model.parameters(), base_optimizer, lr=CONFIG["lr"])

best_loss = float('inf')
obj=True
epoch=0
# Training loop
while obj:  # Number of epochs
    train_loss=0.0
    model.train()  # Set the model to training mode
    for data in tqdm(train_loader):
        images, labels = data[0].to(CONFIG['device']), data[1].to(CONFIG['device'])
        # Forward pass
        classifications,_ = model(images)
        loss = criterion(classifications, labels)
        
        # Backward pass and optimization
        optimizer.zero_grad()
        #optimizer2.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss+=loss.item() * len(data)
    train_loss = train_loss/len(train_loader.sampler)
        #optimizer2.first_step(zero_grad=True)
        
        #criterion(model(images)[0], labels).backward()  # You can use outputs from the first forward pass
        #optimizer2.second_step()
        
    if train_loss < best_loss:
        best_loss = train_loss
        torch.save(model.state_dict(), 'best_model_weights.pth')
        print('Best Model Saved')
    if best_loss < 0.02:
        obj=False
    epoch+=1
        
    print(f'Epoch [{epoch}], Loss: {train_loss:.4f}')'''

'train_dataset = UBCDataset(train_df, transforms=data_transforms[\'N\'])\ntrain_loader = DataLoader(train_dataset, batch_size=CONFIG[\'batch\'], num_workers=2, shuffle=False, pin_memory=True)\n\n\n# Initialize model\nmodel = EfficientNet2(num_classes=CONFIG[\'num_classes\'], threshold=0.3)\n#model = ViTModel(threshold=0.3)\nmodel.to(CONFIG[\'device\']);\nclear_output(wait=True)\n\n# Loss and optimizer\nclass_weights = torch.Tensor(list(weight_balance.values())).to(CONFIG[\'device\'])\n\n# Loss with class weights\ncriterion = nn.CrossEntropyLoss(weight=class_weights)\noptimizer = torch.optim.Adam(model.parameters(), lr=CONFIG["lr"])\n#base_optimizer = torch.optim.Adam\n#optimizer2 = SAM(model.parameters(), base_optimizer, lr=CONFIG["lr"])\n\nbest_loss = float(\'inf\')\nobj=True\nepoch=0\n# Training loop\nwhile obj:  # Number of epochs\n    train_loss=0.0\n    model.train()  # Set the model to training mode\n    for data in tqdm(train_loader):\n        images, labels = data[0].to(CONFIG[

In [20]:
"""import torch
import timm

dfs = []
for (file_path, image_id,label) in zip(test_df["file_path"], test_df["image_id"],test_df['label']):
    dfs.append( get_cropped_images(file_path, image_id,label) )

df_crop = pd.concat(dfs)
df_crop["label"] = 0 # dummy

df_crop = df_crop.drop_duplicates(subset=["image_id", "sx", "ex", "sy", "ey"]).reset_index(drop=True)
test_dataset = UBCDataset2(df_crop, transforms=data_transforms['valid'])
test_loader = DataLoader(test_dataset, batch_size=CONFIG['batch'], num_workers=2, shuffle=False, pin_memory=True)

model = EfficientNet2(num_classes=CONFIG['num_classes'], threshold=0.3)
model.to(CONFIG['device']);
model.load_state_dict(torch.load('/kaggle/input/weights5/best_model_weights3.pth'))
# Test the model
preds = []
with torch.no_grad():
    bar = tqdm(enumerate(test_loader), total=len(test_loader))
    for step, data in bar:        
        images = data[0].to(CONFIG['device'], dtype=torch.float)
        batch_size = images.size(0)

        _, probabilities = model(images)  # Unpack the tuple here
        _, predicted = torch.max(probabilities, 1)  # Use probabilities for predictions
        preds.append(predicted.detach().cpu().numpy())

preds = np.concatenate(preds).flatten()
pred_labels = [label_mapping[label] for label in preds]
df_sub["label"] = pred_labels
df_sub.to_csv("submission.csv", index=False)
print(predicted)"""

'import torch\nimport timm\n\ndfs = []\nfor (file_path, image_id,label) in zip(test_df["file_path"], test_df["image_id"],test_df[\'label\']):\n    dfs.append( get_cropped_images(file_path, image_id,label) )\n\ndf_crop = pd.concat(dfs)\ndf_crop["label"] = 0 # dummy\n\ndf_crop = df_crop.drop_duplicates(subset=["image_id", "sx", "ex", "sy", "ey"]).reset_index(drop=True)\ntest_dataset = UBCDataset2(df_crop, transforms=data_transforms[\'valid\'])\ntest_loader = DataLoader(test_dataset, batch_size=CONFIG[\'batch\'], num_workers=2, shuffle=False, pin_memory=True)\n\nmodel = EfficientNet2(num_classes=CONFIG[\'num_classes\'], threshold=0.3)\nmodel.to(CONFIG[\'device\']);\nmodel.load_state_dict(torch.load(\'/kaggle/input/weights5/best_model_weights3.pth\'))\n# Test the model\npreds = []\nwith torch.no_grad():\n    bar = tqdm(enumerate(test_loader), total=len(test_loader))\n    for step, data in bar:        \n        images = data[0].to(CONFIG[\'device\'], dtype=torch.float)\n        batch_size =

In [21]:
dfs = []
for (file_path, image_id,label) in zip(test_df["file_path"], test_df["image_id"],test_df['label']):
    dfs.append( get_cropped_images(file_path, image_id,label) )

df_crop = pd.concat(dfs)
df_crop["label"] = 0 # dummy

df_crop = df_crop.drop_duplicates(subset=["image_id", "sx", "ex", "sy", "ey"]).reset_index(drop=True)
test_dataset = UBCDataset2(df_crop, transforms=data_transforms['valid'])
test_loader = DataLoader(test_dataset, batch_size=CONFIG['batch'], num_workers=2, shuffle=False, pin_memory=True)

model = EfficientNet2(num_classes=CONFIG['num_classes'], threshold=0.3)
model.to(CONFIG['device']);
model.load_state_dict(torch.load('/kaggle/input/weights5/best_model_weights3.pth'))


preds = []
with torch.no_grad():
    bar = tqdm(enumerate(test_loader), total=len(test_loader))
    for step, data in bar:        
        images = data[0].to(CONFIG["device"], dtype=torch.float)        
        batch_size = images.size(0)
        _, probabilities = model(images)  # Unpack the tuple here
        _, predicted = torch.max(probabilities, 1)  # Use probabilities for predictions
        preds.append(predicted.detach().cpu().numpy())

preds = np.vstack(preds)

for i in range(preds.shape[-1]):
    df_crop[f"cat{i}"] = preds[:, i]

dict_label = {}
for image_id, gdf in df_crop.groupby("image_id"):
    dict_label[image_id] = np.argmax( gdf[ [f"cat{i}" for i in range(preds.shape[-1])] ].values.max(axis=0) )
    #dict_label[image_id] = np.argmax( gdf[ [f"cat{i}" for i in range(preds.shape[-1])] ].values.mean(axis=0) )
preds = np.array( [ dict_label[image_id] for image_id in test_df["image_id"].values ] )

pred_labels = [label_mapping[label] for label in preds]
df_sub["label"] = pred_labels
df_sub.to_csv("submission.csv", index=False)

100%|██████████| 1/1 [00:06<00:00,  6.94s/it]
